In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (138 kB/s)
Reading package lists... Done


In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-10-25 17:40:48--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  3.91MB/s    in 0.2s    

2020-10-25 17:40:49 (3.91 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [ ]:
from pyspark.sql.functions import to_date
from pyspark.sql.types import *
# Read in the Review dataset as a DataFrame

In [ ]:
# Create the vine_table. DataFrame
temp_df = df.select(["review_id", "helpful_votes", "total_votes", "vine", "verified_purchase", df["star_rating"].cast(IntegerType())])
vine_df = temp_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3W4P9UBGNGH1U|          2|            1|          3|   N|                Y|
|R15V54KBMTQWAY|          4|            0|          0|   N|                N|
| RY8I449HNXSVF|          5|            0|          0|   N|                Y|
|R18TLJYCKJFLSR|          5|            0|          0|   N|                Y|
|R1NK26SWS53B8Q|          5|            0|          0|   N|                Y|
|R11LOHEDYJALTN|          5|            0|          0|   N|                Y|
|R3ALQVQB2P9LA7|          4|            0|          0|   N|                Y|
|R3MWLXLNO21PDQ|          5|            0|          0|   N|                Y|
|R2L15IS24CX0LI|          5|            0|          0|   N|                Y|
|R1DJ8976WPWVZU|          3|            0|          0|   N|     

In [ ]:
#filter to only show where total votes are 20 or greater
temp_df = vine_df
total_votes_df = temp_df.filter("total_votes > 19")

In [ ]:
total_votes_count = total_votes_df.agg({"total_votes": "count"}).show()

+------------------+
|count(total_votes)|
+------------------+
|             72587|
+------------------+



In [ ]:
temp1_df = total_votes_df

In [ ]:
# Fitler to show only where helpful vvotes divided by total votes, times 100, the percentage is greater then 50%
temp2_df = temp1_df.withColumn("helpful_votes_percentage_plus50", temp1_df["helpful_votes"] / temp1_df["total_votes"])
#temp2_df.show()
votes_greater50_df = temp2_df.filter("helpful_votes_percentage_plus50 >.5")
votes_greater50_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_votes_percentage_plus50|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|             0.9152542372881356|
|R13VL62Y2HBQ0B|          5|           15|         21|   N|                Y|             0.7142857142857143|
|R22G55KAPZKJQV|          4|           20|         21|   N|                Y|             0.9523809523809523|
|R1610PGTJS7G3N|          2|           28|         44|   N|                Y|             0.6363636363636364|
| RLQL04BL0QXOJ|          4|           45|         47|   N|                Y|             0.9574468085106383|
|R2AYJHH8WJNGAU|          4|           98|        104|   N|                Y|             0.9423076923076923|
|R111DJA10

In [ ]:
votes_greater50_sum_df = votes_greater50_df.agg({"total_votes": "sum"}).withColumnRenamed("sum(total_votes)", "sum_total_votes").show()
votes_greater50_count_df = votes_greater50_df.agg({"total_votes": "count"}).withColumnRenamed("count(total_votes)", "count_total_votes").show()

+---------------+
|sum_total_votes|
+---------------+
|        4150202|
+---------------+

+-----------------+
|count_total_votes|
+-----------------+
|            65384|
+-----------------+



In [ ]:
import pyspark.sql.functions as fn
#votes_greater50_df = votes_greater50_df.crossJoin(votes_greater50_df.groupby().agg(fn.count('total_votes').alias('overall_count')))
#votes_greater50_df.show()

In [ ]:
# Filter to create a vine only df
vine_only_df = votes_greater50_df.filter("vine == 'Y'")
vine_only_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_votes_percentage_plus50|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------------------+
|R1MAOLI5FJHAFM|          4|          249|        261|   Y|                N|             0.9540229885057471|
| R9PYAUDIBJVEC|          4|           12|         22|   Y|                N|             0.5454545454545454|
| R6V9SHMMG5M8F|          5|          101|        110|   Y|                N|             0.9181818181818182|
|R37PVLT6ELL5J4|          4|          181|        209|   Y|                N|             0.8660287081339713|
| R2FSFGWZF24V9|          4|           50|         51|   Y|                N|             0.9803921568627451|
|R3SRW1E8J56IGV|          5|          262|        281|   Y|                N|             0.9323843416370107|
| R86Z11D4

In [ ]:
#Get count of vine only total votes
import pyspark.sql.functions as fn
vine_only_df = vine_only_df.crossJoin(vine_only_df.groupby().agg(fn.count('total_votes').alias('vine_overall_count')))
vine_only_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_votes_percentage_plus50|vine_overall_count|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------------------+------------------+
|R1MAOLI5FJHAFM|          4|          249|        261|   Y|                N|             0.9540229885057471|               613|
| R9PYAUDIBJVEC|          4|           12|         22|   Y|                N|             0.5454545454545454|               613|
| R6V9SHMMG5M8F|          5|          101|        110|   Y|                N|             0.9181818181818182|               613|
|R37PVLT6ELL5J4|          4|          181|        209|   Y|                N|             0.8660287081339713|               613|
| R2FSFGWZF24V9|          4|           50|         51|   Y|                N|             0.98039

In [ ]:
vine_only_count_df = vine_only_df.agg({"total_votes": "count"}).withColumnRenamed("count(total_votes)", "count_vine_total_votes")
vine_only_count_df.show()

+----------------------+
|count_vine_total_votes|
+----------------------+
|                   613|
+----------------------+



In [ ]:
# Shows votes grouped by star rating
vine_only_count_df = vine_only_df.groupby('star_rating').agg({"total_votes": "count"}).withColumnRenamed("count(total_votes)", "count_vine_total_votes")
vine_only_count_df.show()

+-----------+----------------------+
|star_rating|count_vine_total_votes|
+-----------+----------------------+
|          1|                    23|
|          3|                   117|
|          5|                   222|
|          4|                   205|
|          2|                    46|
+-----------+----------------------+



In [ ]:
#Create 5 star only, vine only, df
vine_only_fivestar_only_df = vine_only_df.filter('star_rating == "5"')
vine_only_fivestar_only_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_votes_percentage_plus50|vine_overall_count|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------------------+------------------+
| R6V9SHMMG5M8F|          5|          101|        110|   Y|                N|             0.9181818181818182|               613|
|R3SRW1E8J56IGV|          5|          262|        281|   Y|                N|             0.9323843416370107|               613|
|R2WBPX441TH495|          5|          202|        221|   Y|                N|             0.9140271493212669|               613|
|R2BYBSYHS66ZN8|          5|          109|        114|   Y|                N|              0.956140350877193|               613|
|R2IXC6U7W4OCQ9|          5|          185|        201|   Y|                N|             0.92039

In [ ]:
vine_only_fivestar_only_df = vine_only_fivestar_only_df.groupby().agg({"total_votes": "count"}).withColumnRenamed("count(total_votes)", "count_five_star_total_votes")
vine_only_fivestar_only_df.show()

+---------------------------+
|count_five_star_total_votes|
+---------------------------+
|                        222|
+---------------------------+



In [ ]:
vine_only_fivestar_only2_df = vine_only_fivestar_only_df.crossJoin(vine_only_df.groupby().agg(fn.count('total_votes').alias('vine_five_star_count')))
vine_only_fivestar_only2_df.show()

+---------------------------+--------------------+
|count_five_star_total_votes|vine_five_star_count|
+---------------------------+--------------------+
|                        222|                 613|
+---------------------------+--------------------+



In [ ]:
#Calculate percentage of vine 5 star votes/reviews against total vine reviews
percentage_vinetototal_df = vine_only_fivestar_only2_df.withColumn("vine_percentage", vine_only_fivestar_only2_df["count_five_star_total_votes"] *100/ vine_only_fivestar_only2_df["vine_five_star_count"])
percentage_vinetototal_df.show()


+---------------------------+--------------------+------------------+
|count_five_star_total_votes|vine_five_star_count|   vine_percentage|
+---------------------------+--------------------+------------------+
|                        222|                 613|36.215334420880914|
+---------------------------+--------------------+------------------+



In [ ]:
# create a non-vine review df
notvine_only_df = votes_greater50_df.filter("vine == 'N'")
notvine_only_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_votes_percentage_plus50|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|             0.9152542372881356|
|R13VL62Y2HBQ0B|          5|           15|         21|   N|                Y|             0.7142857142857143|
|R22G55KAPZKJQV|          4|           20|         21|   N|                Y|             0.9523809523809523|
|R1610PGTJS7G3N|          2|           28|         44|   N|                Y|             0.6363636363636364|
| RLQL04BL0QXOJ|          4|           45|         47|   N|                Y|             0.9574468085106383|
|R2AYJHH8WJNGAU|          4|           98|        104|   N|                Y|             0.9423076923076923|
|R111DJA10

In [ ]:
notvine_only_count_df = notvine_only_df.agg({"total_votes": "count"}).withColumnRenamed("count(total_votes)", "count_not_vine_total_votes").show()

+--------------------------+
|count_not_vine_total_votes|
+--------------------------+
|                     64771|
+--------------------------+



In [ ]:
notvine_only_df = notvine_only_df.crossJoin(notvine_only_df.groupby().agg(fn.count('total_votes').alias('notvine_overall_count')))
notvine_only_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------------------+---------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_votes_percentage_plus50|notvine_overall_count|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------------------+---------------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|             0.9152542372881356|                64771|
|R13VL62Y2HBQ0B|          5|           15|         21|   N|                Y|             0.7142857142857143|                64771|
|R22G55KAPZKJQV|          4|           20|         21|   N|                Y|             0.9523809523809523|                64771|
|R1610PGTJS7G3N|          2|           28|         44|   N|                Y|             0.6363636363636364|                64771|
| RLQL04BL0QXOJ|          4|           45|         47|   N|                Y

In [ ]:
notvine_only_count_df = notvine_only_df.agg({"total_votes": "count"}).withColumnRenamed("count(total_votes)", "count_notvine_total_votes")
notvine_only_count_df.show()

+-------------------------+
|count_notvine_total_votes|
+-------------------------+
|                    64771|
+-------------------------+



In [ ]:
# group by star rating to show non-vine 5 star rating count
notvine_only_count_df = notvine_only_df.groupby('star_rating').agg({"total_votes": "count"}).withColumnRenamed("count(total_votes)", "count_notvine_total_votes")
notvine_only_count_df.show()

+-----------+-------------------------+
|star_rating|count_notvine_total_votes|
+-----------+-------------------------+
|          1|                    13932|
|          3|                     5684|
|          5|                    30523|
|          4|                    10491|
|          2|                     4141|
+-----------+-------------------------+



In [ ]:
# Create a non-vine 5 star only df
notvine_only_fivestar_only_df = notvine_only_df.filter('star_rating == "5"')
notvine_only_fivestar_only_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------------------+---------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_votes_percentage_plus50|notvine_overall_count|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------------------+---------------------+
|R13VL62Y2HBQ0B|          5|           15|         21|   N|                Y|             0.7142857142857143|                64771|
|R111DJA10Y6CMU|          5|           26|         39|   N|                Y|             0.6666666666666666|                64771|
|R2EE2TR4MRDV0U|          5|           53|         59|   N|                Y|             0.8983050847457628|                64771|
| RD4A80I5JDHED|          5|           69|         77|   N|                N|             0.8961038961038961|                64771|
|R1D3NR5GREEXXJ|          5|           57|         58|   N|                Y

In [ ]:
notvine_only_fivestar_only_df = notvine_only_fivestar_only_df.groupby().agg({"total_votes": "count"}).withColumnRenamed("count(total_votes)", "count_five_star_total_votes")
notvine_only_fivestar_only_df.show()

+---------------------------+
|count_five_star_total_votes|
+---------------------------+
|                      30523|
+---------------------------+



In [ ]:
notvine_only_fivestar_only2_df = notvine_only_fivestar_only_df.crossJoin(notvine_only_df.groupby().agg(fn.count('total_votes').alias('notvine_five_star_count')))
notvine_only_fivestar_only2_df.show()

+---------------------------+-----------------------+
|count_five_star_total_votes|notvine_five_star_count|
+---------------------------+-----------------------+
|                      30523|                  64771|
+---------------------------+-----------------------+



In [ ]:
# Calculate the percentage of non-vine, five star coutns/reviews against total vine reviews
percentage_notvinetototal_df = notvine_only_fivestar_only2_df.withColumn("notvine_percentage", notvine_only_fivestar_only2_df["count_five_star_total_votes"] *100/ notvine_only_fivestar_only2_df["notvine_five_star_count"])
percentage_notvinetototal_df.show()


+---------------------------+-----------------------+------------------+
|count_five_star_total_votes|notvine_five_star_count|notvine_percentage|
+---------------------------+-----------------------+------------------+
|                      30523|                  64771| 47.12448472310139|
+---------------------------+-----------------------+------------------+

